In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
from peft import PeftModel
import torch

d:\LLMS\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
folder = "./final_model/"
base_model = BertForSequenceClassification.from_pretrained(folder, num_labels=7)
peft_model = PeftModel.from_pretrained(base_model, folder)
tokenizer = BertTokenizer.from_pretrained(folder)
peft_model.eval()

Some weights of the model checkpoint at ./final_model/ were not used when initializing BertForSequenceClassification: ['bert.encoder.layer.0.output.dense.lora_B.default.weight', 'bert.encoder.layer.5.attention.self.value.lora_B.default.weight', 'bert.encoder.layer.6.attention.self.value.lora_A.default.weight', 'bert.encoder.layer.8.attention.self.query.lora_B.default.weight', 'bert.encoder.layer.11.attention.self.query.lora_A.default.weight', 'bert.encoder.layer.7.attention.output.dense.lora_A.default.weight', 'bert.encoder.layer.3.attention.output.dense.lora_B.default.weight', 'bert.encoder.layer.0.attention.output.dense.lora_B.default.weight', 'bert.encoder.layer.5.attention.self.key.lora_B.default.weight', 'bert.encoder.layer.4.intermediate.dense.lora_B.default.weight', 'bert.encoder.layer.8.intermediate.dense.lora_A.default.weight', 'bert.encoder.layer.2.intermediate.dense.lora_B.default.weight', 'bert.encoder.layer.5.attention.output.dense.lora_B.default.weight', 'bert.encoder.lay

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(100000, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(
                    in_features=768, out_features=768, bias=True
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.2, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out

In [4]:
names={'Egyptian':0, 'Iraqi':1, 'Khaleeji':2, 'Levantine':3, 'Lybia':4, 'Maghrebi':5, 'Sudani':6}

In [5]:
input_text = """
والله يا زول، الحاصل شنو كان غريب شديد. أنا كنت ماشي في السوق العربي، لاقيت الزحمة ما طبيعية، ناس كتار وهدوم معلّقة في كل مكان، والدنيا مولعة بالعروض. فجأة لاقيت صاحبي آدم، ما شفتو ليهو مدة، سلمنا واتونسنا، وقال لي تعال نتمشى في بحري. ركبنا ركشة ومشينا، وشفنا ليك شباب بلعبوا كُتشينة في الضل، وناس قاعده على 
البنابر تشرب شاي بالنعناع. الجو كان رهيب، والنسمة حلوة، حسيت براحة. رجعنا البيت متأخر، وأمي زعلت شوية، لكن لما شرحت ليها ضحكت وقالت "المهم إنت بخير
"""
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    padding=True,
    max_length=512,
    truncation=True,
)
inputs = inputs.to(peft_model.device)
with torch.no_grad():
    outputs = peft_model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
names2 = {v: k for k, v in names.items()}
predicted_label = names2[predictions.item()]
print(f"Predicted dialect: {predicted_label}")

Predicted dialect: Sudani
